In [12]:
# SQL 데이터셋 번역기
## 필요한 라이브러리 임포트
import os
import pandas as pd
from openai import OpenAI
from datasets import load_dataset
from pqdm.threads import pqdm  # 병렬 처리 라이브러리
from datasets import Dataset
from dotenv import load_dotenv

from google import genai
from google.genai import types
from pydantic import BaseModel 

# 환경 변수 파일을 로드함. 이는 API 키와 같은 민감한 정보를 소스 코드에 직접 포함시키지 않기 위함임
load_dotenv("./credit-env")

# OpenAI API 클라이언트를 초기화함. 환경 변수에서 API 키를 가져와 사용함
# 이를 통해 GPT-4o 모델에 안전하게 접근할 수 있음
client = OpenAI(api_key=os.getenv("SELF_OPENAI_API_KEY"))

In [5]:
def add_length_column(dataset):
    """
    데이터셋의 텍스트 길이를 분석하여 총 단어 수를 계산하는 함수
    
    Args:
        dataset: Hugging Face 데이터셋 객체
        
    Returns:
        DataFrame: 'total_length' 열이 추가된 pandas DataFrame
    """
    # 데이터셋을 pandas DataFrame으로 변환함
    # 이는 보다 다양한 데이터 조작 기능을 활용하기 위함임
    df = dataset.to_pandas()
    
    # 총 단어 수를 저장할 새로운 열 'total_length'를 초기화함
    # 처음에는 모든 행의 값을 0으로 설정함
    df["total_length"] = 0
    
    # 지정된 열(instruction, input, response)에 대해 반복하여 단어 수를 계산함
    for column_name in ["instruction", "input", "response"]:
        # 각 열의 텍스트 데이터를 문자열로 변환하고, 공백을 기준으로 단어를 분리한 후, 각 리스트의 길이(단어 수)를 계산함
        # 이 과정은 다음과 같이 진행됨:
        # 1. df[column_name] - 해당 열의 데이터를 가져옴
        # 2. .astype(str) - 모든 값을 문자열로 변환함 (비문자열 값이 있을 경우 대비)
        # 3. .str.split() - 각 문자열을 공백 기준으로 분리하여 단어 리스트를 생성함
        # 4. .apply(len) - 각 리스트의 길이(단어 수)를 계산함
        num_words = df[column_name].astype(str).str.split().apply(len)
        
        # 계산된 단어 수를 'total_length' 열에 누적하여 더함
        # 이를 통해 'instruction', 'input', 'response' 열의 총 단어 수를 구할 수 있음
        df["total_length"] += num_words

    # 'total_length' 열이 추가된 DataFrame을 반환함
    return df


def filter_by_total_length(df, difficulty, number_of_samples):
    """
    총 단어 수에 기반하여 난이도별로 데이터를 필터링하는 함수
    
    Args:
        df: 'total_length' 열이 포함된 pandas DataFrame
        difficulty: 난이도 수준 ("easy", "moderate", "difficult" 중 하나)
        number_of_samples: 반환할 최대 샘플 수
        
    Returns:
        DataFrame: 지정된 난이도와 샘플 수에 맞게 필터링된 DataFrame
    """
    # 난이도에 따라 적절한 단어 수 범위로 필터링함
    if difficulty == "easy":
        # 쉬운 난이도: 총 단어 수가 10~100개인 데이터를 선택함
        # .between(10, 100)은 10 <= total_length <= 100 조건을 적용함
        # .iloc[:number_of_samples]는 필터링된 결과에서 지정된 샘플 수만큼만 선택함
        return df[df["total_length"].between(10, 100)].iloc[:number_of_samples]
    elif difficulty == "moderate":
        # 중간 난이도: 총 단어 수가 101~300개인 데이터를 선택함
        return df[df["total_length"].between(101, 300)].iloc[:number_of_samples]
    elif difficulty == "difficult":
        # 어려운 난이도: 총 단어 수가 301~1000개인 데이터를 선택함
        return df[df["total_length"].between(301, 1000)].iloc[:number_of_samples]

In [6]:
## 번역 프롬프트 설정
# 번역을 위한 시스템 프롬프트를 정의함. 이는 AI 모델에게 번역 작업 방식을 지시하는 역할을 함
translation_system_prompt = """
당신은 SQL 관련 지시문을 영어에서 한국어로 번역하는 전문가입니다. 다음 규칙을 반드시 준수하세요:

1. SQL 쿼리 구문(SELECT, FROM, WHERE 등)은 번역하지 않고 원문 그대로 유지합니다.
2. 테이블 이름과 필드 이름(home_team, away_team 등)은 번역하지 않습니다.
3. 지시사항만 자연스러운 한국어로 번역합니다.
4. 불필요한 설명이나 주석은 추가하지 않습니다.
5. 기술 용어는 한국에서 통용되는 용어를 사용합니다.
"""

# 번역을 위한 사용자 프롬프트를 정의함. 이는 실제 번역 요청을 담는 템플릿으로 사용됨
# {instruction}, {input}, {response}는 나중에 실제 데이터로 대체될 포맷 플레이스홀더임
translation_user_prompt = """
다음 SQL 관련 영어 지시문을 한국어로 번역해주세요. 
SQL 구문, 테이블 이름, 필드 이름은 원문 그대로 유지하고 지시사항만 자연스러운 한국어로 번역하세요.

instruction(번역할 내용): {instruction}
input(참고용): {input}
response(참고용): {response}

한국어 번역(instruction만):
"""

## 데이터셋 로드 섹션
# 사용할 데이터셋의 이름과 분할(split)을 지정함
# 'Clinton/Text-to-sql-v1'은 텍스트를 SQL로 변환하는 데이터셋임
dataset_name = "Clinton/Text-to-sql-v1"  # 사용할 데이터셋 이름
split = "train"  # 학습용 데이터셋 분할을 선택함

# Hugging Face 라이브러리를 사용하여 지정된 데이터셋을 로드함
# 로딩 시작을 알리는 메시지를 출력함
print(f"데이터셋 로드 중: {dataset_name} ({split})...")
# load_dataset 함수를 사용하여 지정한 데이터셋과 분할을 로드함
dataset = load_dataset(dataset_name, split=split)

# 이전에 정의된 add_length_column 함수를 사용하여 데이터셋에 총 단어 수를 계산하고 추가함
# 이 함수는 'instruction', 'input', 'response' 열의 텍스트 길이(단어 수)를 합산함
df = add_length_column(dataset)

# 이전에 정의된 filter_by_total_length 함수를 사용하여 난이도별로 데이터를 필터링함
# 단어 수 기준: easy(10-100단어), moderate(101-300단어), difficult(301-1000단어)
# 세 번째 매개변수는 각 난이도에서 선택할 최대 샘플 수를 지정함
# 여기서는 쉬운 난이도와 중간 난이도는 0개(선택하지 않음), 어려운 난이도는 2000개를 선택함
easy = filter_by_total_length(df, "easy", 0)  # 쉬운 난이도 데이터 0개 선택 (사용하지 않음)
medium = filter_by_total_length(df, "moderate", 0)  # 중간 난이도 데이터 0개 선택 (사용하지 않음)
hard = filter_by_total_length(df, "difficult", 2000)  # 어려운 난이도 데이터 2000개 선택

# 필터링된 데이터프레임들을 하나로 합침
# 여기서는 easy와 medium은 0개로 설정했으므로 실질적으로 hard 데이터만 사용됨
dataset = pd.concat([easy, medium, hard])

# 데이터셋을 랜덤하게 섞음(셔플링)
# frac=1은 전체 데이터를 사용함을 의미하고, random_state=42는 재현 가능성을 위한 시드값임
dataset = dataset.sample(frac=1, random_state=42)

# 처리된 pandas DataFrame을 다시 Hugging Face 데이터셋 형식으로 변환함
# 이는 Hugging Face의 기능(예: 배치 처리, 변환 파이프라인)을 활용하기 위함임
dataset = Dataset.from_pandas(dataset)

# 데이터셋 로딩 완료 메시지와 최종 데이터셋 크기를 출력함
print(f"로드 완료! 데이터셋 크기: {len(dataset)} 항목")

데이터셋 로드 중: Clinton/Text-to-sql-v1 (train)...
로드 완료! 데이터셋 크기: 2000 항목


In [7]:
# 데이터셋의 첫 번째 샘플 항목을 출력하여 내용을 확인함
print("\n데이터셋 샘플:")
# 데이터셋의 첫 번째 항목(인덱스 0)을 가져옴
sample = dataset[0]

# 샘플의 모든 키-값 쌍을 순회하며 내용을 출력함
for key, value in sample.items():
    # 값이 문자열이고 길이가 100자를 초과하는 경우, 처음 100자만 표시하고 '...'를 추가함
    # 이는 너무 긴 텍스트가 콘솔을 가득 채우는 것을 방지하기 위함임
    if isinstance(value, str) and len(value) > 100:
        print(f"{key}: {value[:100]}...")
    else:
        # 짧은 텍스트나 문자열이 아닌 값은 전체를 그대로 출력함
        print(f"{key}: {value}")

## 단일 항목 번역 함수 정의
def translate_single_item(item):
    """
    단일 데이터셋 항목을 영어에서 한국어로 번역하는 함수
    
    Args:
        item (dict): 번역할 데이터셋 항목 (instruction, input, response 키를 포함해야 함)
        
    Returns:
        str: 번역된 지시문(instruction) 또는 오류 발생 시 '번역 실패' 메시지
    """
    try:
        # OpenAI API를 호출하여 번역을 수행함
        # GPT-4o 모델을 사용하여 높은 품질의 번역을 얻음
        response = client.chat.completions.create(
            model="gpt-4o",  # 최신 GPT-4o 모델을 사용함
            messages=[
                # 시스템 프롬프트로 번역 규칙을 설정함 (SQL 구문은 번역하지 않음 등)
                {"role": "system", "content": translation_system_prompt},
                # 사용자 프롬프트에 번역할 내용을 포맷팅하여 전달함
                # format 메서드를 사용하여 템플릿의 플레이스홀더를 실제 데이터로 채움
                {"role": "user", "content": translation_user_prompt.format(
                    instruction=item["instruction"],  # 번역할 주요 지시문
                    input=item["input"],             # 참고용 입력 데이터
                    response=item["response"]        # 참고용 응답 데이터
                )}
            ],
            temperature=0.0  # 온도를 0으로 설정하여 일관된 번역 결과를 얻음
        )
        
        # API 응답에서 번역된 텍스트를 추출함
        # choices[0].message.content는 AI 모델이 생성한 응답 텍스트를 가리킴
        # strip() 메서드로 앞뒤 공백을 제거하여 깔끔한 텍스트를 얻음
        translated_instruction = response.choices[0].message.content.strip()
        # 번역된 텍스트를 반환함
        return translated_instruction
    
    # 번역 과정에서 발생할 수 있는 모든 예외를 처리함
    # API 호출 실패, 타임아웃, 권한 오류 등 다양한 오류가 발생할 수 있음
    except Exception as e:
        # 오류 메시지를 출력하여 디버깅을 용이하게 함
        print(f"번역 중 오류 발생: {e}")
        # 오류 발생 시 '번역 실패' 문자열을 반환하여 처리 가능하게 함
        return "번역 실패"

## 번역 테스트 함수 정의
def test_translation(num_samples=3):
    """
    지정된 수의 데이터셋 샘플을 번역하여 테스트하는 함수
    
    Args:
        num_samples (int): 테스트할 샘플 수 (기본값: 3)
    """
    # 테스트 시작을 알리는 메시지 출력
    print(f"\n{num_samples}개 항목 번역 테스트:")
    
    # 지정된 수만큼 데이터셋 항목을 순회함
    # min() 함수를 사용하여 요청한 샘플 수가 데이터셋 크기를 초과하지 않도록 함
    for i in range(min(num_samples, len(dataset))):
        # 현재 인덱스의 데이터셋 항목을 가져옴
        item = dataset[i]
        # 항목 번호를 출력하여 구분함
        print(f"\n항목 {i+1}:")
        # 원본 지시문의 처음 100자를 출력함 (너무 길면 '...'로 표시)
        print(f"원본: {item['instruction'][:100]}...")
        
        # 번역 함수를 호출하여 현재 항목을 번역함
        translated = translate_single_item(item)
        # 번역된 텍스트의 처음 100자를 출력함 (너무 길면 '...'로 표시)
        print(f"번역: {translated[:100]}...")

# 테스트 함수를 호출하여 2개 항목에 대해 번역을 테스트함
# 적은 수로 테스트하는 이유는 API 호출 비용을 절약하고 기능 확인에 충분하기 때문임
test_translation(2)


데이터셋 샘플:
instruction: Top answers in algorithm tag since 2011.
input: CREATE TABLE Tags (
    Id number,
    TagName text,
    Count number,
    ExcerptPostId number,
   ...
response: SELECT Users.DisplayName, COUNT(*) AS UpVotes FROM Tags INNER JOIN PostTags ON PostTags.TagId = Tags...
source: sede
text: Below are sql tables schemas paired with instruction that describes a task. Using valid SQLite, writ...
total_length: 638
__index_level_0__: 15479

2개 항목 번역 테스트:

항목 1:
원본: Top answers in algorithm tag since 2011....
번역: 2011년 이후 알고리즘 태그에서 상위 답변....

항목 2:
원본: A bar chart for what are the number of the order dates of orders with price higher than 1000?...
번역: 가격이 1000보다 높은 주문의 주문 날짜 수에 대한 막대 차트를 만드세요....


In [8]:
def translate_item_for_parallel(item):
    """pqdm 병렬 처리에 사용할 번역 함수"""
    try:
        # 항목 복사 - 원본 데이터를 보존하기 위해 복사본을 만드는 과정임
        result = item.copy()
        
        # API 호출 - OpenAI의 GPT-4o 모델을 사용하여 번역을 수행함
        response = client.chat.completions.create(
            model="gpt-4o",          # 사용할 AI 모델을 지정하는 매개변수임
            messages=[               # 모델에게 전달할 메시지 목록을 구성함
                {"role": "system", "content": translation_system_prompt},  # 시스템 프롬프트로 번역 작업의 기본 지침을 설정함
                {"role": "user", "content": translation_user_prompt.format(  # 사용자 프롬프트를 형식화하여 번역할 내용을 전달함
                    instruction=item["instruction"],  # 지시사항 부분을 전달함
                    input=item["input"],             # 입력 데이터를 전달함
                    response=item["response"]        # 응답 데이터를 전달함
                )}
            ],
            temperature=0.0  # 온도 값을 0으로 설정하여 일관된 번역 결과를 얻도록 함
        )
        
        # 번역된 결과 저장 - API 응답에서 번역된 텍스트를 추출하여 결과 딕셔너리에 저장함
        result["translated_instruction"] = response.choices[0].message.content.strip()
        return result
    
    except Exception as e:
        # 오류 발생 시 원본 반환 (번역 실패 표시 추가) - 예외 처리를 통해 프로그램이 중단되지 않도록 함
        result = item.copy()
        result["translated_instruction"] = f"번역 실패: {str(e)[:100]}..."  # 오류 메시지의 앞부분만 표시하여 가독성을 유지함
        return result

In [9]:
def translate_batch_parallel_pqdm(start_idx=0, batch_size=10, n_jobs=4):
    """pqdm을 사용하여 지정된 범위의 항목을 병렬로 번역함"""
    # 작업 시작 시 기본 정보를 출력하여 사용자에게 작업 범위와 스레드 수를 알려줌
    print(f"\n{batch_size}개 항목 병렬 번역 (시작 인덱스: {start_idx}, 쓰레드 수: {n_jobs}):")
    
    # 번역할 항목 준비 - 전체 데이터셋에서 지정된 범위의 항목들을 추출함
    end_idx = min(start_idx + batch_size, len(dataset))  # 데이터셋 범위를 벗어나지 않도록 끝 인덱스를 조정함
    items_to_translate = [dataset[i] for i in range(start_idx, end_idx)]  # 번역할 항목들의 리스트를 생성함
    
    try:
        # 병렬 처리 시작 전 상태 메시지를 출력하여 사용자에게 진행 상황을 알려줌
        print(f"{n_jobs}개의 쓰레드로 {len(items_to_translate)}개 항목 번역 중...")
        
        # pqdm을 사용한 병렬 처리 - 단 한 줄의 코드로 복잡한 병렬 처리를 구현함
        # items_to_translate: 처리할 항목들의 리스트임
        # translate_item_for_parallel: 각 항목에 적용할 함수임
        # n_jobs: 동시에 실행할 스레드 수를 지정함
        # desc: 진행률 표시줄에 표시될 설명 텍스트임
        translated_items = pqdm(items_to_translate, translate_item_for_parallel, 
                               n_jobs=n_jobs, desc="번역 진행 중")
        
        # 성공/실패 항목 확인 - 번역 결과의 품질을 평가하기 위한 통계를 계산함
        # 번역 실패 항목은 "번역 실패:" 문자열로 시작하므로, 이를 기준으로 성공 항목 수를 계산함
        success_count = sum(1 for item in translated_items if not item["translated_instruction"].startswith("번역 실패"))
        
        # 번역 완료 후 결과 통계를 출력하여 사용자에게 작업 성과를 알려줌
        print(f"배치 번역 완료! 총 {len(translated_items)}개 중 {success_count}개 성공")
        
        # 번역된 항목들의 리스트를 반환하여 추가 처리나 저장이 가능하게 함
        return translated_items
    
    except Exception as e:
        # 전체 병렬 처리 과정에서 예외가 발생한 경우를 처리함
        # 오류 메시지를 출력하여 문제 진단에 도움이 되게 함
        print(f"병렬 처리 실패: {e}")
        
        # 오류 발생 시 빈 리스트를 반환하여 호출자가 오류 상황을 처리할 수 있게 함
        return []

In [38]:
# 병렬 처리로 10개 항목 번역 (4개 프로세스 사용)
translated_batch_parallel = translate_batch_parallel_pqdm(start_idx=0, batch_size=2000, n_jobs=4)

# 번역 결과 확인
for i, item in enumerate(translated_batch_parallel[:3]):  # 처음 3개만 확인
    print(f"\n항목 {i+1}:")
    print(f"원본: {item['instruction'][:100]}...")
    print(f"번역: {item['translated_instruction'][:100]}...")


2000개 항목 병렬 번역 (시작 인덱스: 0, 쓰레드 수: 4):
4개의 쓰레드로 2000개 항목 번역 중...
배치 번역 완료! 총 2000개 중 2000개 성공

항목 1:
원본: Top answers in algorithm tag since 2011....
번역: 2011년 이후 알고리즘 태그에서 상위 답변....

항목 2:
원본: A bar chart for what are the number of the order dates of orders with price higher than 1000?...
번역: 주문 가격이 1000보다 높은 주문의 주문 날짜 수에 대한 막대 차트를 만드세요....

항목 3:
원본: Do you find that PSYCH 405 is more of a challenge than PSYCH 619 ?...
번역: PSYCH 405가 PSYCH 619보다 더 도전적인가요?...


In [39]:
from datasets import Dataset

# 번역이 완료된 딕셔너리 리스트가 있다고 가정합니다
# 예: translated_batch_parallel

# Dataset 객체로 변환
hf_dataset = Dataset.from_list(translated_batch_parallel)

# 데이터셋 정보 확인
print(hf_dataset)
print(f"데이터셋 크기: {len(hf_dataset)}")
print(f"컬럼: {hf_dataset.column_names}")

# 샘플 데이터 확인
print(hf_dataset[0])

Dataset({
    features: ['instruction', 'input', 'response', 'source', 'text', 'total_length', '__index_level_0__', 'translated_instruction'],
    num_rows: 2000
})
데이터셋 크기: 2000
컬럼: ['instruction', 'input', 'response', 'source', 'text', 'total_length', '__index_level_0__', 'translated_instruction']
{'instruction': 'Top answers in algorithm tag since 2011.', 'input': 'CREATE TABLE Tags (\n    Id number,\n    TagName text,\n    Count number,\n    ExcerptPostId number,\n    WikiPostId number\n)\n\nCREATE TABLE PostHistory (\n    Id number,\n    PostHistoryTypeId number,\n    PostId number,\n    RevisionGUID other,\n    CreationDate time,\n    UserId number,\n    UserDisplayName text,\n    Comment text,\n    Text text,\n    ContentLicense text\n)\n\nCREATE TABLE PostTypes (\n    Id number,\n    Name text\n)\n\nCREATE TABLE SuggestedEditVotes (\n    Id number,\n    SuggestedEditId number,\n    UserId number,\n    VoteTypeId number,\n    CreationDate time,\n    TargetUserId number,\n    Tar

# 데이터만들기

In [41]:
# 배치 단위로 처리 (대량 데이터에 효율적)
def process_batch(examples):
    # 배치에 있는 모든 예제에 대해 새 컬럼 생성 - 배치 처리의 결과물을 담을 빈 리스트를 초기화함
    new_texts = []
    
    # 배치 내의 각 항목을 순회하며 처리함 - 배치는 여러 항목의 컬럼별 데이터를 담고 있는 딕셔너리 형태임
    for i in range(len(examples["instruction"])):
        # 번역된 지시사항이 있으면 이를 사용하고, 없으면 원본 지시사항을 사용함 - 데이터 처리 과정의 유연성을 제공함
        instruction = examples["translated_instruction"][i] if "translated_instruction" in examples else examples["instruction"][i]
        
        # 현재 항목의 입력 텍스트와 응답을 가져옴 - 이들은 SQL 쿼리 생성을 위한 컨텍스트로 사용됨
        input_text = examples["input"][i]
        response = examples["response"][i]  # 이 변수는 현재 사용되지 않지만, 나중에 확장 가능하도록 준비된 것으로 보임
        
        # 새로운 입력 텍스트를 구성함 - SQL 쿼리 생성을 위한 프롬프트 형식을 정의함
        new_text = f"""다음은 SQL 테이블 스키마와 수행할 작업에 대한 설명입니다. SQLite 문법에 맞는 쿼리만 작성해주세요. 설명이나 주석 없이 실행 가능한 SQL 쿼리 코드만 출력하세요.
### Instruction: {instruction}
### Input: {input_text}
"""
        
        # 생성된 새 텍스트를 결과 리스트에 추가함 - 모든 항목에 대한 처리 결과를 모음
        new_texts.append(new_text)
    
    # 데이터셋에 새 컬럼 추가 - 원본 데이터를 유지하면서 처리된 결과를 새 컬럼으로 추가함
    examples["new_input"] = new_texts
    
    # 수정된 examples 딕셔너리를 반환함 - Hugging Face 데이터셋의 map 메서드는 이 반환값을 사용하여 데이터셋을 업데이트함
    return examples

# 배치 크기 지정하여 적용 - 데이터셋의 map 메서드로 배치 처리 함수를 적용함
# batched=True: 배치 모드 활성화하여 데이터를 배치 단위로 처리함
# batch_size=100: 한 번에 100개 항목을 처리하여 메모리 효율성과 처리 속도 간의 균형을 맞춤
updated_dataset = hf_dataset.map(process_batch, batched=True, batch_size=100)

Map: 100%|██████████| 2000/2000 [00:00<00:00, 42989.40 examples/s]


In [42]:
updated_dataset

Dataset({
    features: ['instruction', 'input', 'response', 'source', 'text', 'total_length', '__index_level_0__', 'translated_instruction', 'new_input'],
    num_rows: 2000
})

In [43]:
updated_dataset["new_input"][0]

'다음은 SQL 테이블 스키마와 수행할 작업에 대한 설명입니다. SQLite 문법에 맞는 쿼리만 작성해주세요. 설명이나 주석 없이 실행 가능한 SQL 쿼리 코드만 출력하세요.\n### Instruction: 2011년 이후 알고리즘 태그에서 상위 답변.\n### Input: CREATE TABLE Tags (\n    Id number,\n    TagName text,\n    Count number,\n    ExcerptPostId number,\n    WikiPostId number\n)\n\nCREATE TABLE PostHistory (\n    Id number,\n    PostHistoryTypeId number,\n    PostId number,\n    RevisionGUID other,\n    CreationDate time,\n    UserId number,\n    UserDisplayName text,\n    Comment text,\n    Text text,\n    ContentLicense text\n)\n\nCREATE TABLE PostTypes (\n    Id number,\n    Name text\n)\n\nCREATE TABLE SuggestedEditVotes (\n    Id number,\n    SuggestedEditId number,\n    UserId number,\n    VoteTypeId number,\n    CreationDate time,\n    TargetUserId number,\n    TargetRepChange number\n)\n\nCREATE TABLE ReviewTaskResults (\n    Id number,\n    ReviewTaskId number,\n    ReviewTaskResultTypeId number,\n    CreationDate time,\n    RejectionReasonId number,\n    Comment text\n)\n\nC

In [ ]:
updated_dataset.push_to_hub("daje/kotext-to-sql-v1-hard")

# 학습 전 성능 테스트

## vLLM 

vLLM으로 inference를 진행합니다.   
아래 명령어를 별도의 터미널에 실행해주세요. 


```
CUDA_VISIBLE_DEVICES=2 vllm serve "Qwen/Qwen2.5-Coder-7B-Instruct" --tensor-parallel-size 1 --host 0.0.0.0 --port 8100
```

In [51]:
# requests 라이브러리를 가져와 HTTP 요청을 보낼 수 있게 함
import requests

def get_response(input_text):
    # vLLM API 엔드포인트 URL을 지정함 - 로컬 서버의 8100 포트로 설정되어 있음
    url = "http://localhost:8100/v1/chat/completions"

    # HTTP 요청 헤더를 설정함 - Content-Type을 application/json으로 지정하여 JSON 형식의 데이터를 전송함을 서버에 알림
    headers = {
        "Content-Type": "application/json",
    }

    # API 요청에 필요한 데이터를 JSON 형식으로 구성함
    data = {
        "model": "Qwen/Qwen2.5-Coder-7B-Instruct",  # 사용할 모델을 지정함 - Qwen의 코더 특화 모델임
        "messages": [                               # 대화 메시지 배열 구성
            {
                "role": "user",                     # 메시지 역할을 사용자로 지정함
                "content": f"{input_text}"          # 함수 인자로 받은 입력 텍스트를 메시지 내용으로 설정함
            }
        ]
    }

    # POST 요청을 보내고 응답을 받음 - headers와 data를 JSON 형식으로 전송함
    response = requests.post(url, headers=headers, json=data)

    # 응답 상태 코드를 확인하여 성공(200)인 경우와 실패한 경우를 분기 처리함
    if response.status_code == 200:
        # 응답이 성공적이면 JSON으로 파싱하여 결과를 추출함
        result = response.json()
        
        # 주석 처리된 코드: SQL 코드만 추출하는 부분 - 현재는 전체 응답을 반환하도록 변경됨
        # return result["choices"][0]["message"]["content"].split("```sql")[1].split("```")[0]
        
        # 모델의 응답 내용을 반환함 - choices 배열의 첫 번째 항목에서 메시지 내용을 가져옴
        return result["choices"][0]["message"]["content"]
    else:
        # 요청이 실패한 경우 오류 메시지를 생성하여 반환함 - 디버깅을 위한 정보 포함
        return f"Request failed: {response.status_code}, {response.text}"

In [49]:
# 전처리한 데이터를 확인합니다. 
print(updated_dataset['new_input'][1])

다음은 SQL 테이블 스키마와 수행할 작업에 대한 설명입니다. SQLite 문법에 맞는 쿼리만 작성해주세요. 설명이나 주석 없이 실행 가능한 SQL 쿼리 코드만 출력하세요.
### Instruction: 주문 가격이 1000보다 높은 주문의 주문 날짜 수에 대한 막대 차트를 만드세요.
### Input: CREATE TABLE Addresses (
    Address_ID VARCHAR(100),
    Line_1 VARCHAR(255),
    Line_2 VARCHAR(255),
    City_Town VARCHAR(255),
    State_County VARCHAR(255),
    Other_Details VARCHAR(255)
)

CREATE TABLE Ref_Service_Types (
    Service_Type_Code CHAR(15),
    Parent_Service_Type_Code CHAR(15),
    Service_Type_Description VARCHAR(255)
)

CREATE TABLE Clients (
    Client_ID INTEGER,
    Address_ID INTEGER,
    Customer_Email_Address VARCHAR(255),
    Customer_Name VARCHAR(255),
    Customer_Phone VARCHAR(255),
    Other_Details VARCHAR(255)
)

CREATE TABLE Products (
    Product_ID VARCHAR(100),
    Product_Name VARCHAR(255),
    Product_Price DECIMAL(20,4),
    Product_Description VARCHAR(255),
    Other_Product_Service_Details VARCHAR(255)
)

CREATE TABLE Performers (
    Performer_ID INTEGER,
    Address_ID I

In [ ]:
print(updated_dataset['response'][1])

SELECT Order_Date, COUNT(Order_Date) FROM Customer_Orders AS T1 JOIN Order_Items AS T2 ON T1.Order_ID = T2.Order_ID JOIN Products AS T3 ON T2.Product_ID = T3.Product_ID WHERE T3.Product_Price > 1000


1. **문제 이해**: 제품 가격이 1000보다 높은 제품이 포함된 주문들의 날짜별 개수를 구하는 것이 목표입니다.

인공지능이 SQL 쿼리를 작성하기 위해 가장 먼저 해야 할 일은 주어진 문제를 해결하는 데 필요한 관련 테이블들을 식별하는 것입니다.

2. **관련 테이블**: 
이 문제에서는 "주문 가격이 1000보다 높은 주문의 날짜별 개수"를 구해야 하므로:

- **주문 날짜** 정보가 필요 → `Customer_Orders` 테이블 필요
- **제품 가격** 정보가 필요 → `Products` 테이블 필요
- **주문과 제품을 연결**하는 정보 필요 → `Order_Items` 테이블 필요

인공지능은 DB 스키마를 분석하여 어떤 테이블들이 관련이 있는지, 어떤 컬럼들이 필요한지, 그리고 이 테이블들을 어떻게 JOIN해야 하는지 파악해야 합니다. 

제시된 16개의 테이블 중에서 오직 3개의 테이블만이 이 특정 문제를 해결하는 데 관련이 있다는 것을 이해하고, 불필요한 테이블은 제외하는 능력이 중요합니다. 이것이 효율적인 쿼리 작성의 첫 단계입니다.

3. **쿼리 접근법**:
   - 제품 가격이 1000보다 큰 제품 필터링 (WHERE 절 사용)
   - 해당 제품이 포함된 주문 찾기 (JOIN 활용)
   - 주문 날짜별로 그룹화하여 개수 계산 (GROUP BY)

4. **최종 쿼리**: 
   ```sql
   SELECT Order_Date, COUNT(Order_Date) 
   FROM Customer_Orders AS T1 
   JOIN Order_Items AS T2 ON T1.Order_ID = T2.Order_ID 
   JOIN Products AS T3 ON T2.Product_ID = T3.Product_ID 
   WHERE T3.Product_Price > 1000
   ```

In [30]:
print(get_response(updated_dataset['new_input'][1]))


SELECT COUNT(Order_ID) AS Number_of_Orders, Order_Date 
FROM Customer_Orders 
WHERE Order_Price > 1000 
GROUP BY Order_Date;



그러나, Qwen2.5-Coder가 예측한 쿼리는 맞지 않습니다.   

두 가지 중요한 문제가 있습니다:

1. `Customer_Orders` 테이블에는 `Order_Price` 컬럼이 존재하지 않습니다. 이 테이블의 스키마를 보면 주문 가격 정보가 없습니다.

2. 가격 정보는 `Products` 테이블에 있고, 주문-제품 연결 정보는 `Order_Items` 테이블에 있습니다. 따라서 세 테이블을 JOIN해야 합니다.

```sql
SELECT Order_Date, COUNT(Order_Date) 
FROM Customer_Orders AS T1 
JOIN Order_Items AS T2 ON T1.Order_ID = T2.Order_ID 
JOIN Products AS T3 ON T2.Product_ID = T3.Product_ID 
WHERE T3.Product_Price > 1000
```

이 쿼리는 주문과 제품을 연결하고, 제품 가격이 1000보다 큰 경우만 필터링하여 주문 날짜별 개수를 계산합니다. 이처럼 길고 복잡한 문제에 대해서는 7B 모델이 정확히 예측하지 못하는 경향을 보입니다. 

전체 2000개의 데이터에 대해서 성능 테스트를 해보겠습니다. 

In [52]:
from tqdm.auto import tqdm
before_inference = [get_response(temp['new_input']) for temp in tqdm(updated_dataset)]

100%|██████████| 2000/2000 [1:17:26<00:00,  2.32s/it]


In [53]:
len(before_inference)

2000

In [54]:
before_inference[0]

"```sql\nSELECT p.Id, p.Title, p.Body\nFROM Posts p\nJOIN PostTags pt ON p.Id = pt.PostId\nJOIN Tags t ON pt.TagId = t.Id\nWHERE t.TagName LIKE '%algorithm%' AND p.CreationDate > '2011-01-01';\n```"

In [58]:
model_inference_df = pd.DataFrame({
    'inputs': updated_dataset['new_input'],
    'model': before_inference,
    'response': updated_dataset['response']
})
model_inference_df.head()

,inputs,model,response
0,다음은 SQL 테이블 스키마와 수행할 작업에 대한 설명입니다. SQLite 문법에 ...,"```sql\nSELECT p.Id, p.Title, p.Body\nFROM Pos...","SELECT Users.DisplayName, COUNT(*) AS UpVotes ..."
1,다음은 SQL 테이블 스키마와 수행할 작업에 대한 설명입니다. SQLite 문법에 ...,"```sql\nSELECT COUNT(Order_ID) AS Order_Count,...","SELECT Order_Date, COUNT(Order_Date) FROM Cust..."
2,다음은 SQL 테이블 스키마와 수행할 작업에 대한 설명입니다. SQLite 문법에 ...,"```sql\nSELECT c1.name AS course1, c2.name AS ...","SELECT COUNT(*) > 0 FROM course AS COURSE_0, c..."
3,다음은 SQL 테이블 스키마와 수행할 작업에 대한 설명입니다. SQLite 문법에 ...,"```sql\nSELECT p.drug, COUNT(*) AS prescriptio...",SELECT t1.drug FROM (SELECT prescriptions.drug...
4,다음은 SQL 테이블 스키마와 수행할 작업에 대한 설명입니다. SQLite 문법에 ...,```sql\nSELECT EXISTS (\n SELECT 1 \n FR...,SELECT COUNT(*) > 0 FROM procedures_icd WHERE ...


In [60]:
model_inference_df.to_csv("./model_inference_df.csv", index=False)

In [2]:
import pandas as pd

df = pd.read_csv("./data/model_inference_df.csv")
df.head(2)

,inputs,model,response
0,다음은 SQL 테이블 스키마와 수행할 작업에 대한 설명입니다. SQLite 문법에 ...,"```sql\nSELECT p.Id, p.Title, p.Body\nFROM Pos...","SELECT Users.DisplayName, COUNT(*) AS UpVotes ..."
1,다음은 SQL 테이블 스키마와 수행할 작업에 대한 설명입니다. SQLite 문법에 ...,"```sql\nSELECT COUNT(Order_ID) AS Order_Count,...","SELECT Order_Date, COUNT(Order_Date) FROM Cust..."


In [18]:
class json_output(BaseModel):
    reason: str
    score: bool

client = genai.Client(api_key=os.getenv("GOOGLE_API_KEY"))
def evaluation_log(args):
    df, idx = args
    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=f"""너는 주어진 SQL system prompt와 내가 만든 model이 SQL system prompt를 보고 만든 생성결과와 실제 정답을 너에게 전달해줄게. 이를 보고 모델이 알맞게 예측하였는지 평가하는 AI Asisstant입니다.
# SQL system prompt
{df['inputs'].iloc[idx]}
# model 생성결과
{df['model'].iloc[idx]}
# 실제 정답
{df['response'].iloc[idx]}

# 평가 기준
- reason은 한국어로 작성하세요.
-score는 모델이 의미적으로 같은 결과를 조회하는 쿼리라면 True, 아니면 False로 평가하세요.
""",
        config=types.GenerateContentConfig(
        max_output_tokens=1000,  # Set the desired maximum number of output tokens
        temperature=0.0,
        response_mime_type="application/json",
        response_schema=json_output,
        ),
    )
    model_response = response.candidates[0].content.parts[0].text
    return model_response

from pqdm.processes import pqdm
args_list = [(df, idx) for idx in range(len(df))]
result_all_ver1 = pqdm(args_list, evaluation_log, n_jobs=5)

QUEUEING TASKS | : 100%|██████████| 2000/2000 [00:00<00:00, 22321.11it/s]
PROCESSING TASKS | : 100%|██████████| 2000/2000 [10:23<00:00,  3.21it/s]
COLLECTING RESULTS | : 100%|██████████| 2000/2000 [00:00<00:00, 1084219.72it/s]


In [29]:
import json_repair
json_repair_df = pd.DataFrame([json_repair.loads(temp) for temp in result_all_ver1])
json_repair_df.tail(2)


,reason,score
1998,"모델이 비추천 수를 찾는 쿼리를 작성했지만, 실제 정답은 특정 사용자의 게시물에 대...",True
1999,모델은 UC 475 과목이 봄 학기에 개설되는지 확인하는 쿼리를 작성했습니다. 하지...,False


In [31]:
json_repair_df["reason"][1999]

'모델은 UC 475 과목이 봄 학기에 개설되는지 확인하는 쿼리를 작성했습니다. 하지만 실제 정답은 학생 record에 기반하여 UC 475 과목이 봄 학기에 수강 가능한지 확인하는 쿼리입니다. 따라서 모델이 실제 정답과 의미적으로 다른 결과를 조회하는 쿼리를 작성했으므로 score는 false입니다.'

In [32]:
json_repair_df['score'].value_counts()

score
False    1312
True      688
Name: count, dtype: int64

In [33]:
print("score 평균: ", sum(json_repair_df['score']) / len(json_repair_df))

score 평균:  0.344


In [34]:
json_repair_df.to_csv("./data/gemini_evaluation.csv", index=False)